In [20]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import subprocess
import sys
import numpy as np

# Monter Google Drive
drive.mount('/content/drive')

# Accéder au dossier
model_checkpoint = '/content/drive/My Drive/camembert'

# Vérifier le contenu du dossier
try:
    files = os.listdir(model_checkpoint)
    print("Contenu du dossier:", files)
except FileNotFoundError:
    print("Le chemin spécifié est introuvable.")

# Installer la bibliothèque transformers si nécessaire
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
except ModuleNotFoundError:
    print("Transformers not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
    from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Charger le modèle et le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

# Pipeline pour la classification des tokens
ner_pipeline = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"  # Combine les sous-tokens en un seul
)

# Liste des phrases pour l'évaluation
phrases = [
    "Je pars de Paris et j'arrive à Marseille.",
    "Je vais à Bordeaux en partant de Toulouse.",
    "Mon trajet va de VILLIERS SUR LOIR à JARNY.",
    "Nous partons de Lyon et nous arrivons à Nice.",
    "Le départ est prévu à Lille, et l'arrivée se fera à Strasbourg.",
    "Mon itinéraire inclut un départ de Montpellier et une arrivée à Perpignan.",
    "Le train partira de Nantes pour arriver à Rennes.",
    "Un vol de Genève à Zurich est prévu demain matin.",
    "Je vais de Grenoble à Dijon en voiture.",
    "Je voyage de Bruxelles à Amsterdam ce week-end.",
    "L'itinéraire commence à Marseille et se termine à Toulon.",
    "Nous prenons un train de Rouen pour aller au Havre.",
    "Le départ sera à Limoges et l'arrivée à Poitiers.",
    "Je quitte Monaco pour aller à Menton.",
    "Le bus va de Caen à Cherbourg.",
    "Il part de Bayonne et arrive à Biarritz.",
    "Je vais de Chamonix à Annecy.",
    "Nous roulons de Bastia à Ajaccio.",
    "Le ferry part de Calais pour arriver à Douvres.",
    "Je pars de Bordeaux et j'arrive à Paris.",
    "Notre vol va de Madrid à Barcelone.",
    "Un trajet en voiture de Milan à Rome est prévu.",
    "Je vais à Paris en partant de Lyon."
]

# Stocker les entités et leurs types pour l'analyse
entities = []
entity_types = []

# Fonction pour afficher les prédictions
for phrase in phrases:
    print(f"\nPhrase: {phrase}")
    try:
        results = ner_pipeline(phrase)
        for entity in results:
            print(f"Entity: {entity['word']}, Label: {entity['entity_group']}")
            entities.append(entity['word'])
            entity_types.append(entity['entity_group'])
    except Exception as e:
        print(f"Erreur lors du traitement de la phrase '{phrase}': {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contenu du dossier: ['tokenizer_config.json', 'tokenizer.json', 'config.json', 'sentencepiece.bpe.model', 'model.safetensors', 'training_args.bin', 'added_tokens.json', 'special_tokens_map.json', 'checkpoint-2500', 'checkpoint-3000', 'runs', 'checkpoint-500', 'checkpoint-2000', 'checkpoint-1000', 'checkpoint-1500', 'checkpoint-3375']

Phrase: Je pars de Paris et j'arrive à Marseille.
Entity: Paris, Label: DEP
Entity: Marseille, Label: ARR

Phrase: Je vais à Bordeaux en partant de Toulouse.
Entity: Bordeaux, Label: ARR
Entity: Toulouse, Label: DEP

Phrase: Mon trajet va de VILLIERS SUR LOIR à JARNY.
Entity: V, Label: DEP
Entity: ILL, Label: DEP
Entity: IERS SUR LOIR, Label: DEP
Entity: J, Label: ARR
Entity: ARN, Label: ARR
Entity: Y, Label: ARR

Phrase: Nous partons de Lyon et nous arrivons à Nice.
Entity: Lyon, Label: DEP
Entity: Nice, Label: ARR

Phrase: Le 